# Notebook 3 : Unsupervised learning

Notebook prepared by [Chloé-Agathe Azencott](http://cazencott.info), modified by [Victor Laigle](https://eaglev-sci.github.io/).

In this notebook we will explore several techniques for dimension reduction and clustering.

In [ ]:
# Load numpy, pandas and matplotlib (with aliases np, pd and plt respectively)
%matplotlib inline 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', **{'size': 12}) # set the global font size for plots (in pt)

## 1. Principal Components Analysis (PCA)

In this section, we are going to perform a principal component analysis on a dataset describing the scores obtained by the best athletes who participated in a decathlon event in 2004, at the Athens Olympic Games or at the Talence Decastar.

### Data loading

The data are contained in the `decathlon.txt` file.

The file contains 42 rows and 13 columns.

The first line is a header that describes the contents of the columns.

The following lines describe the 41 athletes.

The first 10 columns contain the scores obtained in the different events of the Decathlon.
The 11th column contains the ranking.
The 12th column contains the number of points obtained.
The 13th column contains a categorical variable that specifies the competition concerned (Olympics or Decastar).

We will start by downloading the data from the github repository to your current working directory, and load the data into a `pandas` `dataframe`. 

If you already downloaded the data, you can use the alternative suggested in the next cell.

In [ ]:
!wget https://raw.githubusercontent.com/CBIO-mines/fml-dassault-systems-en/main/data/decathlon.txt

my_data = pd.read_csv('decathlon.txt', sep="\t") 

__Alternatively:__ If you're not on Colab and have already downloaded the file to a `data` folder, uncomment and run the following cell:

In [ ]:
# my_data = pd.read_csv('data/decathlon.txt', sep="\t")  # Read the data into a dataframe

In [ ]:
my_data.head()

### Visualization

A __scatter matrix__ is a visualisation, in k x k panels, of the pairwise relations between k variables:
* on the diagonal, the histogram of each variable
* off-diagonal, the scatter plots between two variables (non standardized)

Your job here is to display the visualization with the [`scatter_matrix`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.scatter_matrix.html) function: 

In [ ]:
from pandas.plotting import scatter_matrix

### START OF YOUR CODE

...

### END OF YOUR CODE

You can also limit the visualization to a few variables for clearer observations. Display the scatter_matrix for the 3 or 4 variables which seem most correlated for example:

In [ ]:
### START OF YOUR CODE

...

### END OF YOUR CODE

Alternatively, the `seaborn` library  la librairie `seaborn` enables more elaborated visualizations than `matplotlib`. You can explore, for instance, the capabilities of [`jointplot`](https://seaborn.pydata.org/generated/seaborn.jointplot.html).

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

sns.jointplot(x='Shot.put', y='400m', data = my_data, 
              height=6, space=0, color='b')

sns.jointplot(x='Rank', y='Points', data = my_data, 
              kind='reg', height=6, space=0, color='b');

We will now perform a principal components analysis of the scores in the 10 events.

Let's start by extracting the predictive variables

In [ ]:
X = np.array(my_data.drop(columns=['Points', 'Rank', 'Competition']))
print(X.shape)

### Data standardization

After visualizing the data, we can notice different data scales and distributions depending on the variables. We therefore reapply the procedure seen in previous labs to standardize our data: we need a `StandardScaler` object contained in the `preprocessing` module of `sklearn`.

In [ ]:
### START OF YOUR CODE

# Import the module
...

In [ ]:
# Create the StandardScaler object
std_scaler = ...

# Fit the object to the data
...

# Transform the data
X_scaled = ...

### END OF YOUR CODE

print(X_scaled.shape)

### Computation of the principal components

`scikit-learn`'s algorithms for matrix factorization are included in the `decomposition` module. For PCA, refer to:
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn import decomposition

Note: We have few variables here so we can afford to calculate all the PCs.

As we've seen in previous labs, most algorithms implemented in `scikit-learn` work as follows:
* we instantiate an object, corresponding to a type of algorithm/model, with its hyperparameters (here the number of components)
* we use the `fit` method to pass the data to this algorithm
* the learned parameters are now accessible as attributes of this object

In [ ]:
# Instantiation of a PCA object with 10 principal components

### START OF YOUR CODE
pca = ...

In [ ]:
# We now pass the standardized data to this object
# This is where the computations are performed
...

### END OF YOUR CODE

### Proportion of variance explained by the PCs

We will now plot the proportion of variance explained by the different components. This is accessible in the attribute `explained_variance_ratio_` of our `pca` object.

In [ ]:
plt.plot(np.arange(1, 11), pca.explained_variance_ratio_, marker='o')

plt.xlabel("Number of principal components")
plt.ylabel("Proportion of variance explained")
plt.show()

We can also display the *cumulative* proportion of variance explained, with the function [`cumsum`](https://numpy.org/doc/2.1/reference/generated/numpy.cumsum.html) from `numpy` (imported above under the alias `np`)

Create a similar graph to the one just above, displaying the cumulative proportion of variance explained as a function of the number of principal components considered.

In [ ]:
### START OF YOUR CODE

...

...
...
...

### END OF YOUR CODE

__Questions:__ 
* What is the proportion of variance explained by the first two components ?
* How many components would we need to use to explain 80% of the data's variance ?

### Data projection on the first two principal components

We will now use only the first two principal components.

Let's start by computing the new data representation, i.e. their projection on these two PCs. 

In [ ]:
X_projected = pca.transform(X_scaled)
print(X_projected.shape)

We can display a scatter plot representing the data along these two PCs.

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1])

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.show()

We can now color each point of the above scatter plot according to the ranking of the athlete it represents.

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1], c=my_data['Rank'])

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.colorbar(label='Rank')
plt.show()

__Question:__ What can we conclude about the interpretation of PC1 ?

### Interpretation of the first two principal components

Each principal component is a linear combination of the variables describing the data. The weights of this linear combination are accessible in `pca.components_`.

We can now visualize, not the individuals as above, but the 10 variables in the space of the 2 PCs.

In [ ]:
pcs = pca.components_
print(pcs[0])

In [ ]:
fig = plt.figure(figsize=(6, 6))

plt.scatter(pcs[0], pcs[1])
for (x_coordinate, y_coordinate, feature_name) in zip(pcs[0], pcs[1], my_data.columns[:10]):
    plt.text(x_coordinate + 0.01, y_coordinate + 0.01, feature_name)                          
    
plt.xlabel("Contribution to PC1")
plt.ylabel("Contribution to PC2")
plt.show()

__Questions:__ 
* Which variables have contributions very similar to the two principal components ?
* What can we deduce from their similarity ?
* How to interpret the sign of the variables' contributions to the _first_ principal component ?

## 2. « Olivetti » data

We will now use dimension reduction to represent, in two dimensions, a dataset of human faces. This is a classic dataset, containing 400 pictures of 64 by 64 pixels. These are pictures of the face of 40 different people, 10 pictures per person, labeled by a class number between 0 and 39 to identify the person.  

We can load this dataset directly thanks to `scikit-learn`:

In [ ]:
from sklearn import datasets

In [ ]:
data = datasets.fetch_olivetti_faces()

__If you do not manage to download the data:__
* Go to : https://github.com/CroncLee/PCA-face-recognition/blob/master/olivetti_py3.pkz
* Download the file (there is a Download button)
* Use the command
```
    data = datasets.fetch_olivetti_faces(data_home="<PATH TO DATA>")
```
Replace "PATH TO DATA" by the path of the folder where you saved the data.

In [ ]:
X = data.data
y = data.target

In [ ]:
print(X.shape)

In [ ]:
print("The dataset contains %d classes" % len(np.unique(y)))

Each image is represented by one value for each pixel (grey scale).

We may visualize these images, under the condition that we reorganize the values (= a vector of length 4096) in 64x64 matrices. For example, for image with index 77:

In [ ]:
plt.imshow(X[77, :].reshape((64, 64)), interpolation='nearest', cmap=plt.cm.gray);

### PCA

Let's start with a principal components analysis as in the previous section:

In [ ]:
pca = decomposition.PCA(n_components=2)
X_transformed_pca = pca.fit_transform(X)

Each picture is now represented, not by 4096 variables, but by 2 variables. We can visualize them on a scatter plot and color them according to their class:

In [ ]:
plt.scatter(X_transformed_pca[:, 0], X_transformed_pca[:, 1], c=y)
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
plt.show()

__Question:__ Do pictures of the same face (= same class) have close or far apart representations ?

We can visualize the contribution of each pixel to the first principal component:

In [ ]:
plt.imshow(pca.components_[0, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray);

And the contribution of each pixel to the second principal component:

In [ ]:
plt.imshow(pca.components_[1, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray);

__Question:__ Which interpretation can we draw from these two images representing the contributions of each pixel to the first two principal components ?

### tSNE

Let's try another dimension reduction method to better separate our classes.

We will use the same approach as for PCA, but with the tSNE algorithm, thanks to the [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class of the `manifold` module.

Now it's your turn to:
- create the `tsne` object from the class cited above. Here we want to reduce our dataset to two dimensions in order to visualize it.
- assign the information of our dataset to this object
- transform our data to obtain new coordinates in two dimensions

In [ ]:
from sklearn import manifold

In [ ]:
### START OF YOUR CODE

tsne = ...
X_transformed = ...

### END OF YOUR CODE

Let's display the result of the tSNE dimension reduction:

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE");

#### Influence of the perplexity parameter

The main hyperparameter influencing the representation obtained by the tSNE algorithm is the "perplexity" parameter. This represents the number of neighbours for which distances are preserved. It therefore influences the preservation of the local structure (low perplexity) or global structure (high perplexity). The representation obtained can vary greatly depending on this parameter.

Test different values of the perplexity parameter and display the corresponding results.

In [ ]:
### START OF YOUR CODE

tsne_low_perp = ...
X_transformed_low_perp = ...

### END OF YOUR CODE

In [ ]:
plt.scatter(X_transformed_low_perp[:, 0], X_transformed_low_perp[:, 1], c=y)
plt.xlabel("First tSNE component")
plt.ylabel("Second tSNE component")
plt.title("tSNE (low perplexity)");

In [ ]:
### START OF YOUR CODE

tsne_high_perp = ...
X_transformed_high_perp = ...

### END OF YOUR CODE

In [ ]:
plt.scatter(X_transformed_high_perp[:, 0], X_transformed_high_perp[:, 1], c=y)
plt.xlabel("First tSNE component")
plt.ylabel("Second tSNE component")
plt.title("tSNE (high perplexity)");

## 3. Clustering

We will now cover another set of unsupervised methods, used to group observations according to their similarities.

We'll start by generating three two-dimensional datasets:
- 4 separated blobs from normal distributions
- 2 nested semi-circles (or "semi-moons")
- 2 concentric circles

In [ ]:
from sklearn import datasets

In [ ]:
# nombre de points
n_samples = 1000

four_blobs, four_blobs_labels = datasets.make_blobs(n_samples=n_samples, centers=4, n_features=2, random_state=170)

moons, moons_labels = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=170)

circles, circles_labels = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05, random_state=170)

First, let's visualize these data:

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 3))

ax[0].scatter(four_blobs[:, 0], four_blobs[:, 1], c=four_blobs_labels, s=20, alpha=0.7, cmap='viridis')
ax[1].scatter(moons[:, 0], moons[:, 1], c=moons_labels, s=20, alpha=0.7, cmap='viridis')
ax[2].scatter(circles[:, 0], circles[:, 1], c=circles_labels, s=20, alpha=0.7, cmap='viridis')

ax[0].set_title('4 blobs')
ax[1].set_title('Moons')
ax[2].set_title('Circles');

Now, let's assume that we do **not** have access to the labels. Which clustering algorithms allow us to recover the clusters corresponding respectively to the four blobs, two moons and two circles ?

### K-means algorithm

The objective of the k-means algorithm is to find $K$ clusters (and their centroid $\mu_k$) so as to **minimize the intra-cluster variance**:

\begin{align}
V = \sum_{k = 1}^{K} \sum_{x \in C_k} \frac{1}{|C_k|} (\|x - \mu_k\|^2)
\end{align}

**Manual implementation**

We'll start by implementing the algorithm "by hand", step by step, to fully understand and visualize what the algorithm does. We will then see how to use directly the K-means implementation in the `sklearn` library. The different steps of the algorithm are:
1. Select the number `k` of clusters (hyperparameter)
2. Initialize the `k` centroids at random among our data points
3. Compute the distances from all data points to these centroids
4. Assign each point to the cluster of the closest centroid
5. Compute the position of the new centroids
6. Repeat steps 3 to 5 until convergence, i.e. until the centroids no longer change from one iteration to the next

We will implement this algorithm on the 4 blobs dataset, starting with the choice of `k` and with the random selection of `k` points from our dataset that will constitute the initial centroids:

In [ ]:
np.random.seed(23)  # set seed. Important for k-means step by step visualisation. With other seeds, it's not as clear how the algorithm works.

k = 4
random_indices = np.random.choice(len(four_blobs), k, replace=False)
centroids_step0 = four_blobs[random_indices]

for i, centroid in enumerate(centroids_step0):
    print(f"Centroid {i} : x = {centroid[0]},\ty = {centroid[1]}")

Let's look at the data with these initial centroids (red crosses)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

ax.scatter(four_blobs[:, 0], four_blobs[:, 1], c='grey', s=20, alpha=0.7)
ax.scatter(centroids_step0[:, 0], centroids_step0[:, 1], c='red', marker='x')

plt.show()

We can now compute the distances between each data point and these centroids

In [ ]:
# Euclidean distance between data point and centroid
def compute_distance(data_point, centroid):
    dist = np.sqrt(np.sum((data_point - centroid)**2))
    return dist

def compute_all_distances(dataset, centroids):
    # Initialize distances array
    distances = np.zeros((k, n_samples))  # k and n_samples defined above
    
    # Calculate distance from each point to each centroid
    for i in range(k):
        for j in range(n_samples):
            distances[i, j] = compute_distance(dataset[j], centroids[i])    

    return distances

In [ ]:
distances = compute_all_distances(four_blobs, centroids_step0)

# Example of distances for the first 5 points to the k centroids
print("Distances :")
print("\t\t", " \t\t".join([f"Point {i+1}" for i in range(5)]))
for i in range(k):
    print(f"Centroid {i}\t", "\t".join(distances[i, :5].astype(str).tolist()))

We now assign to each point the cluster corresponding to the closest centroid. We use the `argmin` function from `numpy` for that. In a way it can be seen as intermediate predicted labels.

In [ ]:
def assign_cluster(distances):
    assignments = np.argmin(distances, axis=0)
    return assignments

intermediate_labels = assign_cluster(distances)
print(intermediate_labels[:5])  # examples of intermediate labels assigned to the data points

We can verify that the intermediate labels assigned indeed correspond to the closest centroid by comparing with the distances calculated above (see previous cell).

Let's visualize these intermediate clusters and the initial centroids on a scatter plot

In [ ]:
def visualise_kmeans(dataset, labels, centroids, ax):

    ax.scatter(dataset[:, 0], dataset[:, 1], c=labels, s=20, alpha=0.7, cmap='viridis')
    ax.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x')
    
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
visualise_kmeans(four_blobs, intermediate_labels, centroids_step0, ax)
plt.show()


We now need to compute the position of the new centroids, which we will plot on a new graph

In [ ]:
def compute_new_centroids(dataset, labels):
    centroids = np.zeros((k, dataset.shape[1]))

    for i in range(k):
        centroids[i] = dataset[labels == i].mean(axis=0)
    
    return centroids

centroids_step1 = compute_new_centroids(four_blobs, intermediate_labels)

fig, axes = plt.subplots(1, 2, figsize=(9, 4))
visualise_kmeans(four_blobs, intermediate_labels, centroids_step0, axes[0])
visualise_kmeans(four_blobs, intermediate_labels, centroids_step1, axes[1])

axes[0].set_title("Initial centroids")
axes[1].set_title("Centroids after one iteration")

plt.show()

We now repeat the different steps: 
- computation of distances to centroids,
- assignment of clusters to data points,
- computation of new centroids

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(24, 4))
n_iter = 10  # Number of iterations of the algorithm

current_fig = 0

centroids = centroids_step1
for i in range(n_iter):
    
    distances = compute_all_distances(four_blobs, centroids)
    intermediate_labels = assign_cluster(distances)

    # We display the visualization every 2 iterations
    if (i+1) % 2 == 0:
        visualise_kmeans(four_blobs, intermediate_labels, centroids, axes[current_fig])
        axes[current_fig].set_title(f"Iteration {i+1}")
        current_fig += 1
        
    centroids = compute_new_centroids(four_blobs, intermediate_labels)

plt.show()


We clearly see here how, through successive iterations, the algorithm is able to correctly identify our clusters, gradually moving the centroids and readjusting the clusters' membership of our data points.

**Question**: In which case(s) can the algorithm yield bad results ?

**Implementation with `sklearn`**

Let's come back to the 3 datasets created earlier (4 blobs, semi-moons and concentric circles), and apply the  to them the `sklearn` implementation of the K-means algorithm, which is an optimized version of the steps we have just seen.

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
from sklearn import cluster

In [ ]:
# START OF YOUR CODE

# Instantiation of the three k-means models with the
# theoretical number of clusters (resp. 4, 2 and 2):
kmeans_four_blobs = ...
kmeans_moons = ...
kmeans_circles = ...

# Application to the data
...
...
...

### END OF YOUR CODE

The `.labels_` attribute contains, for each observation, the number of the cluster to which this observation has been assigned.

In [ ]:
kmeans_four_blobs.labels_[0:10]  # Example of predicted labels for the first 10 data points

Let's visualize what the final clustering looks like for our three datasets:

In [ ]:
# Clustering visualization
fig, ax = plt.subplots(1, 3, figsize=(10, 3))

ax[0].scatter(four_blobs[:, 0], four_blobs[:, 1], c=kmeans_four_blobs.labels_, cmap='viridis')
ax[1].scatter(moons[:, 0], moons[:, 1], c=kmeans_moons.labels_, cmap='viridis')
ax[2].scatter(circles[:, 0], circles[:, 1], c=kmeans_circles.labels_, cmap='viridis')

ax[0].set_title('4 blobs (k=4)')
ax[1].set_title('Moons (k=2)')
ax[2].set_title('Circles (k=2)');

**Questions:**
- Is it the expected clustering ? 
- In which case(s) does the k-means algorithm work correctly ?
- Why doesn't it work in the other case(s) ?

#### Finding $K$ with the silhouette coefficient

It is often the case that the exact number of clusters, $K$, is not known in advance. We can still apply the k-means algorithm and measure the clustering performance to find the best parameter $K$. One of the metrics used is the **silhouette coefficient**.

The silhouette coefficient (or score) enables to **compare the average intra- and inter-cluster distances**:

\begin{align}
\text{score} = \frac{b - a}{\max(a, b)}
\end{align}

with (for each sample):
- $a$ the average intra-cluster distance
- $b$ the average nearest-cluster distance

The score is computed for each observation (with a value between -1 (worst) and 1 (best)), then the average score enables to assess the clustering on the whole point cloud at once.

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score

In [ ]:
from sklearn import metrics

In [ ]:
print(f"4 blobs: Silhouette coefficient for k-means (k=4) : %.2f" % 
      metrics.silhouette_score(four_blobs, kmeans_four_blobs.labels_))
print(f"Moons: Silhouette coefficient for  k-means (k=2) : %.2f" % 
      metrics.silhouette_score(moons, kmeans_moons.labels_))
print(f"Circles: Silhouette coefficient for  k-means (k=2) : %.2f" % 
      metrics.silhouette_score(circles, kmeans_circles.labels_))


Let's try to evaluate the performance of the clustering as a function of the number of clusters $K$, by varying it in the range [2, .., 8]

In [ ]:
k_values = range(2, 9)
names = ['4 blobs', 'Moons', 'Circles']
fig, ax = plt.subplots(2, 3, figsize=(16, 10))

for i, dataset in enumerate([four_blobs, moons, circles]):
    silhouettes = []
    
    for kval in k_values:

        ### START OF YOUR CODE

        # Initialize a KMeans model with the number of cluster tested:
        kmeans_k = ...
        
        # Train the model on the data
        ...
        
        # Append the silhouette score obtained to the list
        ...
        
        ### END OF YOUR CODE
    
    # Visualization of the silhouette score
    ax[0,i].plot(k_values, silhouettes)
    ax[0,i].set_xlabel("K")
    ax[0,i].set_ylabel("Silhouette score")
    ax[0,i].set_title(names[i])
    
    print("Dataset:", names[i])
    best_silhouette = np.max(silhouettes)
    print("Optimal silhouette coefficient: %.2f" % best_silhouette)
    best_K = k_values[silhouettes.index(best_silhouette)]
    print("Corresponding number of cluster K: %.0f" % best_K)
    
    # Final clustering with the best K
    kmeans_k = cluster.KMeans(n_clusters=best_K)
    kmeans_k.fit(dataset)
    ax[1,i].scatter(dataset[:, 0], dataset[:, 1], c=kmeans_k.labels_, cmap='viridis')
    ax[1,i].set_xlabel('x1')
    ax[1,i].set_ylabel('x2')
    ax[1,i].set_title('Clustering with ' + str(best_K) + ' clusters')
fig.tight_layout()


**Conclusions:** 
- The k-means algorithm provides a satisfactory clustering for the dataset with four well-separated blobs, even without knowing the ideal number of clusters in advance, in which case the silhouette coefficient allows to find this ideal number.
- However, despite optimizing the silhouette score, this algorithm does not provide good results for the other datasets, whether for the two nested moons or the two concentric circles.

We will therefore now try another clustering algorithm and test its performance to compare it to k-means. We will limit ourselves to the two datasets for which the k-means algorithm does not work.

### DBSCAN (Density-based clustering)

The DBSCAN (Density-Based Spatial Clustering of Applications with Noise) algorithm works in two steps:
- All sufficiently close observations are connected to each other.
- Observations with a minimum number of connected neighbors are considered *core samples*, from which the clusters are expanded. **All observations sufficiently close to a *core sample* belong to the same cluster as this one**.

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

The DBSCAN algorithm takes two hyperparameters as input:
- `eps`: the *size of the neighborhood*, in other words, the distance between two data points below which one point is considered inside the neighborhood of the other.
- `min_samples`: the minimum number of neighbors for a data point to be considered a *core sample*.

In [ ]:
### START OF YOUR CODE

# Initialization of the two DBSCAN models
# with hyperparameters eps=0.2, min_samples=2:
dbscan_moons = ...
dbscan_circles = ...

# Fit to the data
...
...

### END OF YOUR CODE


Again, the `.labels_` attribute contains, for each observation, the number of the cluster to which this observation has been assigned.

In [ ]:
print("Number of labels for the moons dataset:", len(np.unique(dbscan_moons.labels_)))
print("Number of labels for the circles dataset:", len(np.unique(dbscan_circles.labels_)))

Let's visualize the clusters obtained:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].scatter(moons[:, 0], moons[:, 1], c=dbscan_moons.labels_, cmap='viridis')
ax[0].set_title("Clustering DBSCAN (eps=0.2)")

ax[1].scatter(circles[:, 0], circles[:, 1], c=dbscan_circles.labels_, cmap='viridis')
ax[1].set_title("Clustering DBSCAN (eps=0.2)");

We can see here that the DBSCAN algorithm is able to identify the two respective clusters in the two datasets that were problematic for the k-means algorithm.

We can also note that we did not need to provide a number of clusters beforehand for the algorithm to correctly identify the right number of clusters. However, the algorithm is sensitive to the two hyperparameters mentioned above, which we will now evaluate.

#### Role of the neighborhood size parameter (`eps`)

We will assess the influence of the `eps` parameter on the concentric circles dataset (`circles`)

In [ ]:
### START OF YOUR CODE

# Choose low and high values for eps, for example 0.05 and 2.0
eps_low = ...
eps_high = ...

# Instantiation of a DBSCAN clustering object with low eps and another one with high eps
dbscan_low = ...
dbscan_high = ...

# Fit to the data
...
...

### END OF YOUR CODE

**Question:** What is the number of clusters obtained in our two models ? (Use the `.labels` attribute)

In [ ]:
### START OF YOUR CODE

...
...

### END OF YOUR CODE

In [ ]:
fig = plt.figure(figsize=(5, 5))

outliers = np.where(dbscan_low.labels_ == -1)[0]
plt.scatter(circles[outliers, 0], circles[outliers, 1], marker='*', color='red')

non_outliers = np.where(dbscan_low.labels_ != -1)[0]
plt.scatter(circles[non_outliers, 0], circles[non_outliers, 1], c=dbscan_low.labels_[non_outliers])
plt.title(f"Clustering DBSCAN (eps={eps_low})");

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles[:, 0], circles[:, 1], c=dbscan_high.labels_)
plt.title(f"Clustering DBSCAN (eps={eps_high})");

#### Trouver eps avec le coefficient de silhouette

In [ ]:
print("Silhouette coefficient for DBSCAN (eps=0.2) : %.2f" % 
      metrics.silhouette_score(circles, dbscan_circles.labels_))

In [ ]:
eps_values = np.logspace(-3, 1, 40)
silhouettes = []

for eps in eps_values:
    dbscan_eps = cluster.DBSCAN(eps=eps, min_samples=2)
    dbscan_eps.fit(circles)
    if len(np.unique(dbscan_eps.labels_)) > 1:  # Needed to compute the silhouette coeff
        silhouettes.append(metrics.silhouette_score(circles, dbscan_eps.labels_))
    else:
        silhouettes.append(-1)

In [ ]:
plt.plot(eps_values, silhouettes)
plt.xscale("log")
plt.xlabel("eps (log scale)")
plt.ylabel("silhouette");

In [ ]:
best_silhouette = np.max(silhouettes)
print(f"Optimal silhouette coefficient: {best_silhouette:.2f}")
print(f"Corresponding eps: {eps_values[silhouettes.index(best_silhouette)]:.2f}")

Let's see what the clustering looks like using the `eps` parameter yielding the best silhouette coefficient:

In [ ]:
best_eps = eps_values[silhouettes.index(best_silhouette)]

dbscan_best = cluster.DBSCAN(eps=best_eps, min_samples=2)
dbscan_best.fit(circles)

fig = plt.figure(figsize=(5, 5))
plt.scatter(circles[:, 0], circles[:, 1], c=dbscan_best.labels_)
plt.title(f"Clustering DBSCAN (eps={best_eps:.2f})");

**Question:** 
- What is the problem here ?
- Is the silhouette coefficient adapted to our dataset ? 

### Adjusted Rand Index

The adjusted Rand index enables to **compare a clustering's result with labels**. For each pair of observations, we look at whether they belong to the same cluster or not, in the predicted and the true clustering. The index takes values between 0 (random clustering) and 1 (perfect clustering).

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html

__Question:__ Why not use an evaluation metric from classification models here ?

In [ ]:
print("Adjusted Rand Index of K-means (K=2) : %.2f" % metrics.adjusted_rand_score(circles_labels, kmeans_circles.labels_))

In [ ]:
print("Adjusted Rand Index of DBSCAN (eps=0.2) : %.2f" % metrics.adjusted_rand_score(circles_labels, dbscan_circles.labels_))

## Bonus: Clustering on Penguins

The goal here is to test several unsupervised clustering methods on a new dataset, and to compare them. Try the methods seen above such as [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) or [sklearn.cluster.DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). You can also try other methods such as Gaussian mixtures ([sklearn.mixture.GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)).

Which methods do you think should work better? Why?

The dataset we suggest you use here concerns different species of penguins and some physical characteristics. The 3 species are: Adelie penguins, Gentoo penguins and Chinstrap penguins.

In [ ]:
!wget https://raw.githubusercontent.com/CBIO-mines/fml-dassault-systems-en/main/data/penguins_data.csv

palmerpenguins = pd.read_csv("penguins_data.csv")

__Alternatively:__ If you already downloaded the data, uncomment the following line:

In [ ]:
# palmerpenguins = pd.read_csv("data/penguins_data.csv")

In [ ]:
palmerpenguins.head()

The first step is to filter some data to avoid missing values (NA or NaN).

In [ ]:
palmerpenguins = palmerpenguins[palmerpenguins['bill_depth_mm'].notna()]
palmerpenguins = palmerpenguins.reset_index(drop=True)

We will focus only on two characteristics of our penguins: beak length and body mass

In [ ]:
penguins_X = np.array(palmerpenguins[["bill_length_mm", "body_mass_g"]])

In [ ]:
from sklearn import preprocessing

Our variables have very different scales, as we can see in the data displayed above. Therefore, we need to standardize our data.

In [ ]:
# Standardization (centering and scaling)
penguins_X = ...

In [ ]:
species_names, species_int = np.unique(palmerpenguins.species, return_inverse=True)
penguins_labels = species_int
species_names

Let's first visualize our data on a graph:

In [ ]:
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels)
plt.xlabel("bill_length_mm (scaled)")
plt.ylabel("body_mass_g (scaled)");

It's now up to you to test different clustering algorithms on these data, and evaluate their performance. Will you be able to obtain a perfect clustering ?

In addition to the clustering algorithms used above, you can try Gaussian mixtures ([GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)) for example, or other methods from the `cluster` or `mixture` modules. For each, provide the silhouette coefficients and adjusted Rand indices you obtain, as well as a visualization of the resulting clustering.

### KMeans

In [ ]:
# Initialization of a k-means model with k=3
kmeans = ...

# Fit to the data
...

In [ ]:
# plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=kmeans.labels_, marker='*')


plt.xlabel("bill_length_mm (scaled)")
plt.ylabel("body_mass_g (scaled)");

In [ ]:
print("Silhouette coefficient for k-means (k=3) : %.2f" % metrics.silhouette_score(penguins_X, kmeans.labels_))

In [ ]:
print("Adjusted Rand Index for K-means (K=3) : %.2f" % metrics.adjusted_rand_score(penguins_labels, kmeans.labels_))

### DBSCAN

In [ ]:
eps_values = np.logspace(-3, 1, 40)
silhouettes = []

for eps in eps_values:
    dbscan_eps = ...
    ...
    if len(np.unique(dbscan_eps.labels_)) > 1: # needed to compute silhouette coeff
        silhouettes.append(...)
    else:
        silhouettes.append(-1)

In [ ]:
plt.plot(eps_values, silhouettes)
plt.xscale("log")
plt.xlabel("eps (log scale)")
plt.ylabel("silhouette");

In [ ]:
best_silhouette = ...
print("Optimal silhouette coefficient: %.2f" % best_silhouette)
best_eps = ...
print("Corresponding eps: %.2f" % best_eps)

Retrieve the eps and min_samples parameters from the optimal DBSCAN model, create a new model with these and fit it to the data.

In [ ]:
dbscan_opt = ...
...

In [ ]:
np.unique(dbscan_opt.labels_)

In [ ]:
print("Adjusted Rand Index for DBSCAN : %.2f" % metrics.adjusted_rand_score(penguins_labels, dbscan_opt.labels_))

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=dbscan_opt.labels_, marker='*')


plt.xlabel("bill_length_mm (scaled)")
plt.ylabel("body_mass_g (scaled)");

### Gaussian mixture model

The Gaussian Mixture Model **optimizes the parameters of a finite number of Gaussians** to fit the data.

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html

In [ ]:
from sklearn import mixture

In [ ]:
# Initialization of Gaussian Mixture model with 3 components
gmm = ...

# Fit to the data
...

# Clusters prediction
gmm_labels = ...

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=gmm_labels, marker='*')


plt.xlabel("bill_length_mm (scaled)")
plt.ylabel("body_mass_g (scaled)");

In [ ]:
print("Silhouette coefficient for GMM (k=3) : %.2f" % metrics.silhouette_score(penguins_X, gmm_labels))

In [ ]:
print("Adjusted Rand Index for GMM (K=3) : %.2f" % metrics.adjusted_rand_score(penguins_labels, gmm_labels))